In [1]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

# 设置设备：MacBook 推荐使用 mps，如不可用则使用 cuda 或 cpu
device = torch.device("mps" if torch.backends.mps.is_available() else 
                      "cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# 数据预处理与加载
# ---------------------------
# 训练集数据预处理：resize 到256, 随机裁剪 224×224, 随机水平翻转, 转为 Tensor, 并归一化
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# 测试集数据预处理：resize 到256, CenterCrop 到224×224, 转为 Tensor, 并归一化
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 下载并加载 CIFAR100 数据集
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset  = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# ---------------------------
# 模型加载与修改
# ---------------------------
# 加载预训练的 AlexNet 模型
model = models.alexnet(pretrained=True)
# 修改最后一层全连接层，使其输出 100 类
num_features = model.classifier[6].in_features  # 原本为4096
model.classifier[6] = nn.Linear(num_features, 100)
model = model.to(device)

# 定义损失函数与优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ---------------------------
# 结构化剪枝操作
# ---------------------------
# 针对 model.features 中的所有卷积层，采用 L1 范数进行结构化剪枝（剪除 20% 的输出通道）
for name, module in model.features.named_modules():
    if isinstance(module, nn.Conv2d):
        prune.ln_structured(module, name='weight', amount=0.2, n=1, dim=0)
        # 移除剪枝时增加的 reparameterization，使剪枝结果永久生效
        prune.remove(module, 'weight')
        print(f"Pruned layer: {name}")

# ---------------------------
# 微调训练
# ---------------------------
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    start_time = time.time()
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct_train / total_train
    
    # 在测试集上评估模型
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_test_loss += loss.item()
            _, predicted = outputs.max(1)
            total_test += targets.size(0)
            correct_test += predicted.eq(targets).sum().item()
    
    test_loss = running_test_loss / len(test_loader)
    test_acc = 100. * correct_test / total_test
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, "
          f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%, "
          f"Epoch Time: {epoch_time:.2f}s")

print("Fine-tuning finished.")

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# 加载 CIFAR100 测试集（train=False），数据会自动下载到 './data' 目录
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

with torch.no_grad():
    for i, (inputs, _) in enumerate(data_loader):
        inputs = inputs.to(device)
        _ = model(inputs)
        if i >= 10:
            break
    
# 测量推断时间：对前 num_samples 个样本进行时间测量，并计算平均推断时间
num_samples = 100
total_time = 0.0
with torch.no_grad():
    for i, (inputs, _) in enumerate(data_loader):
        if i >= num_samples:
            break
        inputs = inputs.to(device)
        # 对于 mps 设备，目前无需手动同步，直接计时即可
        start_time = time.time()
        _ = model(inputs)
        end_time = time.time()
        total_time += (end_time - start_time)
    
    
average_time = total_time / num_samples
print("Average inference time over {} samples in {}: {:.6f} seconds".format(num_samples, model.__class__.__name__, average_time))

Using device: mps
Files already downloaded and verified
Files already downloaded and verified


/Users/dtjgp/miniconda3/envs/greenai/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dtjgp/miniconda3/envs/greenai/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Pruned layer: 0
Pruned layer: 3
Pruned layer: 6
Pruned layer: 8
Pruned layer: 10
Epoch 1/10: Train Loss: 2.897, Train Acc: 28.33%, Test Loss: 1.914, Test Acc: 47.60%, Epoch Time: 93.49s
Epoch 2/10: Train Loss: 1.828, Train Acc: 49.65%, Test Loss: 1.639, Test Acc: 55.06%, Epoch Time: 89.03s
Epoch 3/10: Train Loss: 1.532, Train Acc: 56.78%, Test Loss: 1.396, Test Acc: 60.65%, Epoch Time: 88.92s
Epoch 4/10: Train Loss: 1.350, Train Acc: 61.12%, Test Loss: 1.322, Test Acc: 62.35%, Epoch Time: 89.05s
Epoch 5/10: Train Loss: 1.231, Train Acc: 64.13%, Test Loss: 1.272, Test Acc: 63.74%, Epoch Time: 89.16s
Epoch 6/10: Train Loss: 1.122, Train Acc: 66.90%, Test Loss: 1.195, Test Acc: 65.58%, Epoch Time: 88.59s
Epoch 7/10: Train Loss: 1.042, Train Acc: 68.70%, Test Loss: 1.170, Test Acc: 66.33%, Epoch Time: 89.00s
Epoch 8/10: Train Loss: 0.970, Train Acc: 70.73%, Test Loss: 1.155, Test Acc: 66.88%, Epoch Time: 88.88s
Epoch 9/10: Train Loss: 0.899, Train Acc: 72.74%, Test Loss: 1.123, Test Acc: 6

In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import datasets
import time

# 设置设备：优先使用 mps（适用于 MacBook），否则使用 cuda 或 cpu
device = torch.device("mps" if torch.backends.mps.is_available() else 
                      "cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------------------
# 数据预处理与加载
# ---------------------------
# 训练集数据预处理：resize 到256, 随机裁剪 224×224, 随机水平翻转, 转为 Tensor, 并归一化
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# 测试集数据预处理：resize 到256, CenterCrop 到224×224, 转为 Tensor, 并归一化
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 下载并加载 CIFAR100 数据集
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset  = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# ---------------------------
# 模型加载与修改（使用 ResNet18）
# ---------------------------
model = models.resnet18(pretrained=True)
# 修改最后一层全连接层，使其输出 100 类
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 100)
model = model.to(device)

# 定义损失函数与优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# ---------------------------
# 定义辅助函数：统计模型参数量
# ---------------------------
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    nonzero_params = sum(torch.count_nonzero(p).item() for p in model.parameters())
    return total_params, nonzero_params

# 打印剪枝前模型的参数统计信息
total_before, nonzero_before = get_model_size(model)
print(f"Before pruning: Total params: {total_before}, Nonzero params: {nonzero_before}, Sparsity: {100*(1 - nonzero_before/total_before):.2f}%")

# ---------------------------
# 结构化剪枝操作
# ---------------------------
# 对模型中所有的卷积层进行结构化剪枝（剪除 20% 的输出通道）
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        prune.ln_structured(module, name='weight', amount=0.2, n=1, dim=0)
        # 移除剪枝时增加的 reparameterization，使剪枝结果永久生效
        prune.remove(module, 'weight')
        print(f"Pruned layer: {name}")

# 打印剪枝后模型的参数统计信息
total_after, nonzero_after = get_model_size(model)
print(f"After pruning: Total params: {total_after}, Nonzero params: {nonzero_after}, Sparsity: {100*(1 - nonzero_after/total_after):.2f}%")

# ---------------------------
# 微调训练
# ---------------------------
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    start_time = time.time()
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct_train / total_train
    
    # 在测试集上评估模型
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_test_loss += loss.item()
            _, predicted = outputs.max(1)
            total_test += targets.size(0)
            correct_test += predicted.eq(targets).sum().item()
    
    test_loss = running_test_loss / len(test_loader)
    test_acc = 100. * correct_test / total_test
    
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, "
          f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%, "
          f"Epoch Time: {epoch_time:.2f}s")

print("Fine-tuning finished.")

# ---------------------------
# 推断时间测试（取部分测试样本）
# ---------------------------
# 使用与测试集相同的预处理加载测试数据
inference_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)
data_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

# 预热部分样本
with torch.no_grad():
    for i, (inputs, _) in enumerate(data_loader):
        inputs = inputs.to(device)
        _ = model(inputs)
        if i >= 10:
            break

# 测量推断时间：对前 num_samples 个样本进行时间测量，并计算平均推断时间
num_samples = 100
total_time = 0.0
with torch.no_grad():
    for i, (inputs, _) in enumerate(data_loader):
        if i >= num_samples:
            break
        inputs = inputs.to(device)
        start_time = time.time()
        _ = model(inputs)
        end_time = time.time()
        total_time += (end_time - start_time)
    
average_time = total_time / num_samples
print("Average inference time over {} samples in {}: {:.6f} seconds".format(num_samples, model.__class__.__name__, average_time))

Using device: mps
Files already downloaded and verified
Files already downloaded and verified


/Users/dtjgp/miniconda3/envs/greenai/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dtjgp/miniconda3/envs/greenai/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Before pruning: Total params: 11227812, Nonzero params: 11227812, Sparsity: 0.00%
Pruned layer: conv1
Pruned layer: layer1.0.conv1
Pruned layer: layer1.0.conv2
Pruned layer: layer1.1.conv1
Pruned layer: layer1.1.conv2
Pruned layer: layer2.0.conv1
Pruned layer: layer2.0.conv2
Pruned layer: layer2.0.downsample.0
Pruned layer: layer2.1.conv1
Pruned layer: layer2.1.conv2
Pruned layer: layer3.0.conv1
Pruned layer: layer3.0.conv2
Pruned layer: layer3.0.downsample.0
Pruned layer: layer3.1.conv1
Pruned layer: layer3.1.conv2
Pruned layer: layer4.0.conv1
Pruned layer: layer4.0.conv2
Pruned layer: layer4.0.downsample.0
Pruned layer: layer4.1.conv1
Pruned layer: layer4.1.conv2
After pruning: Total params: 11227812, Nonzero params: 9000493, Sparsity: 19.84%
Epoch 1/10: Train Loss: 2.571, Train Acc: 38.93%, Test Loss: 1.600, Test Acc: 57.59%, Epoch Time: 156.28s
Epoch 2/10: Train Loss: 1.393, Train Acc: 62.76%, Test Loss: 1.175, Test Acc: 66.73%, Epoch Time: 152.65s
Epoch 3/10: Train Loss: 1.065, Tr